In [ ]:
%set_env LD_LIBRARY_PATH=llama.cpp/ggml/src:llama.cpp/src
import threading
import subprocess

In [ ]:
model = "Llama-3.2-1B-Instruct"
# model = "Meta-Llama-3.1-8B-Instruct"
# model = "Qwen2.5-1.5B-Instruct"
# model = "gemma-2-2b-it"
# model = "Phi-3.5-mini-instruct"
# model = "Phi-3-medium-128k-instruct"
# model = "Mistral-Nemo-Instruct-2407"
# model = "Mistral-Small-Instruct-2409"
# size = "IQ4_XS"
# size = "Q5_K_M"
size = "Q8_0"
path = 'eval_results/' + model.replace('.', '-') + '-' + size + '/'

## MMLU-Pro

In [ ]:
# pkill -9 llama-server
subprocess.run(['pkill', '-9', 'llama-server'])

In [ ]:
# llama-server -m models/gemma2/gemma-2-2b-it-Q4_K_S.gguf --ctx-size 6144 --n-gpu-layers 99 --flash-attn --parallel 2
## It seems that there is a noticeable benefit when the degree of parallelism is 2,
## and there is also some benefit when it is 4,
## but beyond 4, the benefits are not significant.
slot_ctx = 3072
parallel = 4
ngl = '99'
# the default host and port
host = '127.0.0.1'
port = '8080'

server = threading.Thread(target=lambda: subprocess.run([
    'llama.cpp/bin/llama-server',
    '-m', "models/" + model + "-" + size + ".gguf",
    '--ctx-size', str(parallel * slot_ctx),
    '--parallel', str(parallel),
    '--host', host,
    '--port', port,
    '--n-gpu-layers', ngl,
    '--flash-attn'], check=True), daemon=True)

server.start()

In [ ]:
# python run_openai.py --url http://localhost:8080/v1 --model gguf --dataset TIGER-Lab/MMLU-Pro --category 'computer science' --parallel 2 --style multi_chat_zeroshot
# in 4 minutes 18 seconds for Llama-3.2-1B-Instruct-Q8_0 on RTX 3080
style = 'multi_chat_zeroshot'
category = 'computer science'
subprocess.run([
    'python', 'run_openai.py',
    '--url', 'http://' + host + ':' + port + '/v1',
    '--model', model + "-" + size,
    '--dataset', 'TIGER-Lab/MMLU-Pro',
    '--parallel', str(parallel),
    '--category', category,
    '--style', style], check=True)

subprocess.run(['mv', path + category + '_result.json', path + category + '_result.full.json'])
subprocess.run(['mv', path + category + '_summary.json', path + category + '_summary.full.json'])
subprocess.run(['mv', path + 'report.txt', path + 'cs-report.txt'])

In [ ]:
# python run_openai.py --url http://localhost:8080/v1 --model gguf --dataset TIGER-Lab/MMLU-Pro --category 'engineering' --parallel 2 --style multi_chat_zeroshot
# in 21 minutes 43 seconds for Llama-3.2-1B-Instruct-Q8_0 on RTX 3080
style = 'multi_chat_zeroshot'
category = 'engineering'
subprocess.run([
    'python', 'run_openai.py',
    '--url', 'http://' + host + ':' + port + '/v1',
    '--model', model + "-" + size,
    '--dataset', 'TIGER-Lab/MMLU-Pro',
    '--parallel', str(parallel),
    '--category', category,
    '--style', style], check=True)

subprocess.run(['mv', path + category + '_result.json', path + category + '_result.full.json'])
subprocess.run(['mv', path + category + '_summary.json', path + category + '_summary.full.json'])
subprocess.run(['mv', path + 'report.txt', path + 'engi-report.txt'])

In [ ]:
# python run_openai.py --url http://localhost:8080/v1 --model gguf --dataset sam-paech/mmlu-pro-nomath-sml --parallel 2 --style multi_chat_zeroshot
# in 16 minutes 52 seconds for Llama-3.2-1B-Instruct-Q8_0 on RTX 3080
style = 'multi_chat_zeroshot'
subprocess.run([
    'python', 'run_openai.py',
    '--url', 'http://' + host + ':' + port + '/v1',
    '--model', model + "-" + size,
    '--dataset', 'sam-paech/mmlu-pro-nomath-sml',
    '--parallel', str(parallel),
    '--style', style], check=True)

subprocess.run(['mv', path + 'report.txt', path + 'mmlu-pro-nomath-sml-report.txt'])

## openai/MMMLU

In [ ]:
# pkill -9 llama-server
subprocess.run(['pkill', '-9', 'llama-server'])

In [ ]:
slot_ctx = 2900
parallel = 3
ngl = '99'
# the default host and port
host = '127.0.0.1'
port = '8080'

server = threading.Thread(target=lambda: subprocess.run([
    'llama.cpp/bin/llama-server',
    '-m', "models/" + model + "-" + size + ".gguf",
    '--ctx-size', str(parallel * slot_ctx),
    '--parallel', str(parallel),
    '--host', host,
    '--port', port,
    '--n-gpu-layers', ngl,
    '--flash-attn'], check=True), daemon=True)

server.start()

In [ ]:
# python run_openai-mmmlu.py --url http://localhost:8080/v1 --model gguf --dataset openai/MMMLU --category ES_LA
mmmlu_es = threading.Thread(target=lambda: subprocess.run([
    'python', 'run_openai-mmmlu.py',
    '--url', 'http://' + host + ':' + port + '/v1',
    '--model', model + "-" + size,
    '--dataset', 'openai/MMMLU',
    '--category', 'ES_LA'], check=True), daemon=True)

mmmlu_es.start()

In [ ]:
# python run_openai-mmmlu.py --url http://localhost:8080/v1 --model gguf --dataset openai/MMMLU --category FR_FR
mmmlu_fr = threading.Thread(target=lambda: subprocess.run([
    'python', 'run_openai-mmmlu.py',
    '--url', 'http://' + host + ':' + port + '/v1',
    '--model', model + "-" + size,
    '--dataset', 'openai/MMMLU',
    '--category', 'FR_FR'], check=True), daemon=True)

mmmlu_fr.start()

In [ ]:
# python run_openai-mmmlu.py --url http://localhost:8080/v1 --model gguf --dataset openai/MMMLU --category ZH_CN
mmmlu_zh = threading.Thread(target=lambda: subprocess.run([
    'python', 'run_openai-mmmlu.py',
    '--url', 'http://' + host + ':' + port + '/v1',
    '--model', model + "-" + size,
    '--dataset', 'openai/MMMLU',
    '--category', 'ZH_CN'], check=True), daemon=True)

mmmlu_zh.start()

In [ ]:
# in 9 minutes 13 seconds for Llama-3.2-1B-Instruct-Q8_0 on RTX 3080
subprocess.run(['mv', path + 'report.txt', path + 'mmmlu-report.txt'])

In [ ]:
### ATTENTION: NOT include in our customized benchmark for saving time
# python run_openai-mmmlu.py --url http://localhost:8080/v1 --model gguf --dataset openai/MMMLU --category DE_DE
mmmlu_de = threading.Thread(target=lambda: subprocess.run([
    'python', 'run_openai-mmmlu.py',
    '--url', 'http://' + host + ':' + port + '/v1',
    '--model', model + "-" + size,
    '--dataset', 'openai/MMMLU',
    '--category', 'DE_DE'], check=True), daemon=True)

mmmlu_de.start()

In [ ]:
### ATTENTION: NOT include in our customized benchmark for saving time
# python run_openai-mmmlu.py --url http://localhost:8080/v1 --model gguf --dataset openai/MMMLU --category PT_BR
mmmlu_pt = threading.Thread(target=lambda: subprocess.run([
    'python', 'run_openai-mmmlu.py',
    '--url', 'http://' + host + ':' + port + '/v1',
    '--model', model + "-" + size,
    '--dataset', 'openai/MMMLU',
    '--category', 'PT_BR'], check=True), daemon=True)

mmmlu_pt.start()

## Perplexity

In [ ]:
# pkill -9 llama-server
subprocess.run(['pkill', '-9', 'llama-server'])

In [ ]:
# about 2 minutes for Llama-3.2-1B-Instruct-Q8_0 on RTX 3080
ngl = '99'
with open(path + 'perplexity.log', 'w+') as outfile:
    subprocess.run([
    'llama.cpp/bin/llama-perplexity',
    '-m', "models/" + model + "-" + size + ".gguf",
    '-f', 'wikitext-2-raw/wiki.test.raw',
    '--n-gpu-layers', ngl], stdout = outfile, stderr=subprocess.STDOUT)